In [ ]:
from src.Evaluation.Evaluator import EvaluatorHoldout
import scipy.sparse as sps
import random
import os
import numpy as np
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

URM_train = sps.load_npz('Dataset/URM_train.npz')
URM_test_complete = sps.load_npz('Dataset/URM_test.npz')

URM_all=URM_train + URM_test_complete

In [ ]:
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
def objective(trial):
    topk=trial.suggest_int('topk',15,500,log=True)
    alpha=trial.suggest_float('alpha',0.05,2,log=True)
    normalize_similarity=trial.suggest_categorical('normalize_similarity',[True,False])
    recommender=P3alphaRecommender(URM_train)
    recommender.fit(topK=topk,alpha=alpha,implicit=True,normalize_similarity=normalize_similarity)
    result_df, _ = evaluator.evaluateRecommender(recommender)
    return result_df.loc[10, "MAP"]

In [ ]:
import optuna
from optuna.samplers import RandomSampler

database_path='hypertuning_results/tuning_p3aplha.db'

study=optuna.create_study(direction='maximize',
    study_name='hyperparameters_tuning_p3alpha',load_if_exists=True,
    storage=f'sqlite:///{database_path}',sampler=RandomSampler(seed=SEED))

study.optimize(objective, n_trials=150, n_jobs=5)